<a href="https://colab.research.google.com/github/Ekaterina-Kostina/study/blob/ML/Kostina_hw4_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Предисловие
При выполнении этой домашней работы я вдохновилась двумя вещами:
1) тетрадкой на Kaggle с применением RNN для генерации текстов Тейлор Свифт (https://www.kaggle.com/code/modmari/taylor-swift-s-song-lyrics-generator)
2) датасетом песее Эминема (https://www.kaggle.com/datasets/aditya2803/eminem-lyrics)

Скомибинровав эти 2 вещи, я решила поэкспериментировать с генерацией строк для ИИ-Эминема....

### Сбор данных: используйте готовый или соберите свой корпус в формате plain text для генерации текстов

In [1]:
pip install dm-sonnet -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.4/268.4 kB 16.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import sonnet as snt
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv("Eminem_Lyrics.csv", sep='\t', comment='#', encoding = "ISO-8859-1")
df.head()

,Album_Name,Song_Name,Lyrics,Album_URL,Views,Release_date,Unnamed: 6
0,Music To Be Murdered By: Side B,Alfred (Intro),"[Intro: Alfred Hitchcock]\nThus far, this albu...",https://genius.com/albums/Eminem/Music-to-be-m...,24.3K,"December 18, 2020",NaN
1,Music To Be Murdered By: Side B,Black Magic,"[Chorus: Skylar Grey & Eminem]\nBlack magic, n...",https://genius.com/albums/Eminem/Music-to-be-m...,180.6K,"December 18, 2020",NaN
2,Music To Be Murdered By: Side B,Alfreds Theme,"[Verse 1]\nBefore I check the mic (Check, chec...",https://genius.com/albums/Eminem/Music-to-be-m...,285.6K,"December 18, 2020",NaN
3,Music To Be Murdered By: Side B,Tone Deaf,"[Intro]\nYeah, I'm sorry (Huh?)\nWhat did you ...",https://genius.com/albums/Eminem/Music-to-be-m...,210.9K,"December 18, 2020",NaN
4,Music To Be Murdered By: Side B,Book of Rhymes,"[Intro]\nI don't smile, I don't frown, get too...",https://genius.com/albums/Eminem/Music-to-be-m...,193.3K,"December 18, 2020",NaN


In [6]:
df.describe()

,Album_Name,Song_Name,Lyrics,Album_URL,Views,Release_date,Unnamed: 6
count,348,348,348,348,330,340,1
unique,18,304,310,18,264,86,1
top,Unreleased Songs,Stan,"[Intro: Mark Avery & Eddie]\nMeet Eddie, 23 ye...",https://genius.com/albums/Eminem/Unreleased-songs,1.2M,"February 23, 1999","May 23, 2000"
freq,39,3,3,39,7,23,1


In [7]:
lyrics_list = df['Lyrics'].tolist()
lyrics_list[:3]

['[Intro: Alfred Hitchcock]\nThus far, this album has provided musical accompaniment to make your passing pleasant\nOur next number is designed to drown out the sound of shovels\nMusic to be buried by',
 '[Chorus: Skylar Grey & Eminem]\nBlack magic, night walker (Yeah)\nShe haunts me like no other (Feel like)\nNobody told me (I don\'t know) love is pain, oh (I know we just met)\nBlack magic, dark water (But it\'s like)\nSurrounds me like no other (It\'s like I know you better than anyone)\nShe\'s got my heart in chains\n\n[Verse 1: Eminem]\nWe\'re volatile, I can\'t call it, though\nIt\'s like too large a peg, and too small a hole (Yeah)\nBut she cheats and I catch her like the common cold (Sneeze)\nLast time, I broke her collar bone, she\'s intolerabl? (Yeah)\nAll I know is that the sex is ph?nomenal, though\nWe\'re an unlikely pair (Yeah)\nLike two different Nike Airs\nBut I\'m the same size she wears (Jordans)\nSo I think we\'re soulmates though (Yeah)\nPeople don\'t like us togethe

### Предварительный анализ: чистка текста

In [8]:
import re

cleaned_lyrics_list = []
for song_lyrics in lyrics_list:
    no_brackets = re.sub(r'\[.*?\]', '', song_lyrics)
    no_slashes = re.sub(r'/', ' ', no_brackets)
    lines = no_slashes.split('\n')
    lines = [line for line in lines if line.strip() != '']
    cleaned_lyrics_list.extend(lines)

cleaned_lyrics_list[:3]

['Thus far, this album has provided musical accompaniment to make your passing pleasant',
 'Our next number is designed to drown out the sound of shovels',
 'Music to be buried by']

In [9]:
len(cleaned_lyrics_list)

29320

Сократим датасет до 1/4, чтобы не тратить много ресурсов нашего Colab На обучение и не превысить лимиты...

In [10]:
corp = cleaned_lyrics_list[:len(cleaned_lyrics_list)//4]

In [11]:
corpus = [line.lower() for line in corp]

print(f"{len(corpus)} — lines of text\n")
print(f"The first 5 lines:\n")
print(corpus[:20])

7330 — lines of text

The first 5 lines:

['thus far, this album has provided musical accompaniment to make your passing pleasant', 'our next number is designed to drown out the sound of shovels', 'music to be buried by', 'black magic, night walker (yeah)', 'she haunts me like no other (feel like)', "nobody told me (i don't know) love is pain, oh (i know we just met)", "black magic, dark water (but it's like)", "surrounds me like no other (it's like i know you better than anyone)", "she's got my heart in chains", "we're volatile, i can't call it, though", "it's like too large a peg, and too small a hole (yeah)", 'but she cheats and i catch her like the common cold (sneeze)', "last time, i broke her collar bone, she's intolerabl? (yeah)", 'all i know is that the sex is ph?nomenal, though', "we're an unlikely pair (yeah)", 'like two different nike airs', "but i'm the same size she wears (jordans)", "so i think we're soulmates though (yeah)", "people don't like us together, but like we ca

### Обучение модели. Используйте образец из туториала по RNN

In [12]:
# Инициализируем токенизатор
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)

In [13]:
tokenizer.texts_to_sequences([corpus[0]])[0]

[3612, 440, 18, 327, 314, 3613, 3614, 3615, 6, 109, 21, 3616, 3617]

In [14]:
X = []
y = []
MAX_SAMPLES = 30000

for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])
        if len(X) >= MAX_SAMPLES:
            break
    if len(X) >= MAX_SAMPLES:
        break

In [15]:
class RNN(snt.RNNCore):

  # Конструктор класса, инициализирует параметры RNN
  def __init__(self, hidden_size, activation=tf.tanh, name="vanilla_rnn"):
    """
    hidden_size: размер скрытого слоя
    activation: тип функции активации
    name: название модели
    """
    # Вызываем конструктор родительского класса (snt.RNNCore)
    super(RNN, self).__init__(name=name)

    # Сохраняем размер скрытого состояния (количество нейронов в скрытом слое)
    self._hidden_size = hidden_size

    # Сохраняем функцию активации (по умолчанию используется гиперболический тангенс)
    self._activation = activation

In [16]:
  def _build(self, input_, prev_state):
    """
    input_: тензор с текущим x_t
    prev_tate: тезор с h_{t-1}
    """
    # Создаем линейный слой для преобразования входных данных в скрытое состояние
    self._in_to_hidden_linear = snt.Linear(
        self._hidden_size, name="in_to_hidden")

    # Создаем линейный слой для преобразования предыдущего скрытого состояния в новое
    self._hidden_to_hidden_linear = snt.Linear(
        self._hidden_size, name="hidden_to_hidden")

    # Применяем линейное преобразование к входным данным
    in_to_hidden = self._in_to_hidden_linear(input_)

    # Применяем линейное преобразование к предыдущему скрытому состоянию
    hidden_to_hidden = self._hidden_to_hidden_linear(prev_state)

    # Складываем результаты и применяем функцию активации
    output = self._activation(in_to_hidden + hidden_to_hidden)

    # Возвращаем выходное значение и новое скрытое состояние (они одинаковы в этой реализации)
    return output, output

  # Свойство state_size возвращает размерность скрытого состояния
  @property
  def state_size(self):
    return tf.TensorShape([self._hidden_size])

  # Свойство output_size возвращает размерность выходного значения
  @property
  def output_size(self):
    return tf.TensorShape([self._hidden_size])

In [20]:
max_sequence_len = max([len(x) for x in X])
X = pad_sequences(X, maxlen=max_sequence_len, padding='pre')
y = np.array(y)
y = to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [21]:
# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_len))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
history = model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.0294 - loss: 7.2595 - val_accuracy: 0.0365 - val_loss: 6.9435
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.0389 - loss: 6.3699 - val_accuracy: 0.0372 - val_loss: 7.0385
Epoch 3/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.0454 - loss: 6.1506 - val_accuracy: 0.0408 - val_loss: 7.0125
Epoch 4/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0532 - loss: 5.9720 - val_accuracy: 0.0498 - val_loss: 7.0087
Epoch 5/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0743 - loss: 5.7421 - val_accuracy: 0.0643 - val_loss: 6.9601
Epoch 6/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0933 - loss: 5.4867 - val_accuracy: 0.0730 - val_loss: 6.9639
Epoch 7/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.1158 - loss: 5.1988 - val_accuracy: 0.0717 - val_loss: 6.9943
Epoch 8/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.1365 - loss: 4.9845 - val_accuracy: 0

In [23]:

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (64, 25, 100)          │       727,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (64, 150)              │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (64, 7275)             │     1,098,525 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,929,877 (22.62 MB)

 Trainable params: 1,976,625 (7.54 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,953,252 (15.08 MB)

### Генерация текста. Используйте образец из туториала по RNN

In [25]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)

        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += ' ' + output_word
    return seed_text

generated = generate_text("slim", 25, max_sequence_len)
print(generated)


slim sha mind's in the gutter but insane you wanna see her with her with her lucifer white face mask face carnage up no white flags


### Сгенерируйте несколько текстов с помощью созданной модели

In [27]:
# генерируем несколько текстов, используя предыдущую функцию
def generate_multiple_texts(seed_texts, next_words, max_sequence_len):
    generated_texts = []
    for seed_text in seed_texts:
        generated_text = generate_text(seed_text, next_words, max_sequence_len)
        generated_texts.append(generated_text)
    return generated_texts

seed_texts = [
    "slim",
    "I am not",
    "Eminem",
    "Rap",
    "Music",
    "You better",
    "Christina"
]

generated_texts = generate_multiple_texts(seed_texts, next_words=25, max_sequence_len=max_sequence_len)

for i, generated_text in enumerate(generated_texts):
    print(f"Generated text {i+1}:\n{generated_text}\n")


Generated text 1:
slim sha mind's in the gutter but insane you wanna see her with her with her lucifer white face mask face carnage up no white flags

Generated text 2:
I am not coming back i done told over 'em comment on shit i don't havin' regrets yeah oh oh oh oh oh oh oh oh oh oh

Generated text 3:
Eminem i just keep droppin' shit i saw you i'm too fly now yeah oh oh oh oh oh oh oh oh oh oh oh yeah

Generated text 4:
Rap circles around 'round clowns surround sound sound the internet thought i feed off of you yeah yeah you on my elevator uh huh yeah yeah

Generated text 5:
Music to be buried by the pen exploded yeah like a storm screamin' out no regrets yeah no apologies i ain't takin' shit damn oh oh

Generated text 6:
You better you better run cocksucker haha and i get involved in a workplace shooting dozens of officers and damn damn look to put started to load

Generated text 7:
Christina down compose yourself mozart the sides of the asylum wall art and i'm at my clown face but 

### Выводы
- Я экспериментировала с разным количеством max tokens И длиной датасета, иногда мне не хватало ОЗУ, а иногда данных было слишком мало и Accuracy не доходило даже до 70%. Сейчас тренировочная accuracy почти 80%, но валидационная оцень низкая (примерно 16% и 10 val_loss).
- Вероятно, модель переобучилась (?) или стоит еще настроить гиперпараметры
- В целом, полученный результат выглядит правдоподобно для текстов в стилитике Эминема: есть гиперболы в его стиле, после токена Slim закономерно идет shady, и даже есть «Music to be buried by», что отсылает к альбому «Music to Be Murdered By», есть даже какие-то попытки в рифму
- При увеличении контекста (длины строки до 30, 40, 50) RNN теряет контекст и начинает генерировать совсем несвязный бред, либо зацикливате yeah/oh oh oh до бесконечностп...